In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [35]:
#Imports
from google import genai
from tavily import TavilyClient
import os
import re
import json
import sqlite3
from datetime import datetime
from typing import List, Dict, Any, Optional
import numpy as np
from pathlib import Path
from typing import Callable, Any, Dict, Optional
from memory_system import (
    MemoryStore, MemoryRetriever, FeedbackHandler, 
    ContextBuilder, AgentMemoryIntegration
)
from reflection_system import ReflectionWorkflow

In [ ]:
# Create client
client = genai.Client(api_key=os.environ.get('GOOGLE_API_KEY'))
model="gemini-2.5-pro"
tavily_client = TavilyClient(api_key=os.environ.get('TAVILY_API_KEY')) # Initialize Tavily client

# Memory

# Initialize memory system
memory_store = MemoryStore("pm_agent_memory.db")
retriever = MemoryRetriever(memory_store)
feedback_handler = FeedbackHandler(memory_store)
context_builder = ContextBuilder(retriever)
memory_integration = AgentMemoryIntegration(
    memory_store, retriever, feedback_handler, context_builder
)

In [38]:
#Agents
class ProductManagerAgent:
    """Product Manager Agent with conversation memory and long-term learning."""
    
    def __init__(self, model: str, client, memory_integration: AgentMemoryIntegration, num_reflections: int = 0):
        self.model = model
        self.client = client
        self.memory_integration = memory_integration
        self.conversation_history = []
        self.num_reflections = num_reflections
        self.reflection_workflow = ReflectionWorkflow(client, model, num_reflections)
        self.system_instruction = """You are an experienced Product Manager with expertise in:
        - Product strategy and roadmap planning
        - Market research and competitive analysis
        - User research and requirements gathering
        - Feature prioritization and backlog management
        - Stakeholder communication
        - Metrics and KPI definition
        - Go-to-market strategies
        - Agile methodologies
        - Providing feedback to your product specialist, engineering, design, data science, testing, and marketing teams

        You provide practical, actionable advice and ask clarifying questions when needed.
        When provided with search results or past memories, incorporate relevant information into your response and cite sources."""
    
    def __call__(self, user_query: str, num_reflections: Optional[int] = None, verbose: bool = False):
        """
        Make the agent callable like a function.
        
        Args:
            user_query: The user's query
            num_reflections: Number of reflections for this specific query (overrides default)
            verbose: If True, returns reflection data
            
        Returns:
            (response, memory_id) tuple or (response, memory_id, reflection_data) if verbose
        """
        # Use query-specific reflections or fall back to instance default
        reflections_to_use = num_reflections if num_reflections is not None else self.num_reflections
        
        # Get memory context
        memory_context = self.memory_integration.pre_run_hook(user_query, "pm_interaction")

        # Decide if search is needed
        decision_prompt = f"""Analyze this query and determine if you need web search.

Query: {user_query}

Respond with ONLY "YES" or "NO":"""

        decision_response = self.client.models.generate_content(
            model=self.model,
            contents=decision_prompt
        )
        
        needs_search = "YES" in decision_response.text.strip().upper()
        
        # Perform search if needed
        search_context = ""
        if needs_search:
            print("🔍 Searching the web...")
            search_results = search_web(user_query)
            if isinstance(search_results, list):
                search_context = "\n\nWeb Search Results:\n"
                for i, result in enumerate(search_results, 1):
                    search_context += f"\n{i}. {result.get('title', '')}\n{result.get('content', '')}\n"
        
        # Build conversation context
        conversation_context = ""
        if self.conversation_history:
            conversation_context = "\n\nPrevious conversation:\n"
            for entry in self.conversation_history[-5:]:  # Keep last 5 exchanges
                conversation_context += f"User: {entry['user']}\nAssistant: {entry['assistant']}\n\n"

        # Define response generation function for reflection workflow
        def generate_response(query: str) -> str:
            full_query = (
                memory_context +
                conversation_context + 
                f"\n{query}" +
                search_context
            )
            
            response = self.client.models.generate_content(
                model=self.model,
                contents=full_query,
                config={'system_instruction': self.system_instruction}
            )
            return response.text
        
        # Execute with or without reflection
        if reflections_to_use > 0:
            print(f"🤔 Reflecting {reflections_to_use} time(s)...")
            # Temporarily update reflection workflow
            self.reflection_workflow.num_reflections = reflections_to_use
            result = self.reflection_workflow.execute(
                initial_query=user_query,
                generate_response_fn=generate_response,
                system_instruction=self.system_instruction
            )
            final_response = result['final_response']
            reflection_data = result
        else:
            final_response = generate_response(user_query)
            reflection_data = None

        # Store in conversation history
        self.conversation_history.append({
            'user': user_query,
            'assistant': final_response
        })
        
        # Log to long-term memory
        memory_id = self.memory_integration.post_run_hook(
            user_query,
            final_response,
            memory_type="pm_interaction",
            metadata={
                'search_used': needs_search, 
                'num_reflections': reflections_to_use, 
                'reflection_used': reflections_to_use > 0
            }
        )

        if verbose and reflection_data:
            return final_response, memory_id, reflection_data
        
        return final_response, memory_id
    
    def set_reflections(self, num_reflections: int):
        """Update the default number of reflections."""
        self.num_reflections = num_reflections
        self.reflection_workflow.num_reflections = num_reflections
        print(f"Default reflection count updated to: {num_reflections}")
    
    def apply_feedback(self, memory_id: int, feedback_type: str, 
                      feedback_content: str = ""):
        """Apply feedback to a specific interaction."""
        self.memory_integration.apply_feedback(memory_id, feedback_type, feedback_content)
        print(f"Feedback '{feedback_type}' applied to memory {memory_id}")
        
    def clear_history(self):
        """Clear conversation history."""
        self.conversation_history = []
        self.reflection_workflow.clear_history()
        print("Conversation history cleared.")

In [39]:
#Helper functions
def remove_markdown(response_text):
    """Remove markdown formatting and print clean text."""
    # Remove markdown formatting
    text = re.sub(r'\*\*(.+?)\*\*', r'\1', response_text)  # Bold
    text = re.sub(r'\*(.+?)\*', r'\1', text)  # Italic
    text = re.sub(r'`(.+?)`', r'\1', text)  # Code
    text = re.sub(r'#+\s', '', text)  # Headers
    return text
def chat_with_agent(agent, agent_name="Agent"):
    """
    Generic interactive chat interface for any agent.
    
    Args:
        agent: The agent instance (callable) to chat with
        agent_name: Display name for the agent (default: "Agent")
    
    Type 'exit', 'quit', or 'bye' to end the conversation.
    Type 'clear' to clear conversation history.
    """
    print("=" * 60)
    print(f"{agent_name} Chat")
    print("=" * 60)
    print(f"Start chatting with the {agent_name}!")
    print("Type 'exit', 'quit', or 'bye' to end the chat.")
    print("Type 'clear' to clear conversation history.\n")
    
    while True:
        user_input = input("You: ").strip()
        
        if user_input.lower() in ['exit', 'quit', 'bye']:
            print(f"\n{agent_name}: Thanks for chatting! Goodbye!")
            break
        
        if user_input.lower() == 'clear':
            if hasattr(agent, 'clear_history'):
                agent.clear_history()
            continue
            
        if not user_input:
            continue
            
        try:
            response = agent(user_input)
            print(f"\n{agent_name}: {remove_markdown(response)}\n")
        except Exception as e:
            print(f"\nError: {e}\n")
            print("Please try again or type 'exit' to quit.\n")
def search_web(query):
    """Search the web using Tavily."""
    try:
        response = tavily_client.search(query, max_results=5)
        return response['results']
    except Exception as e:
        return f"Search error: {e}"



In [40]:
# Create agent instances
pm_agent = ProductManagerAgent(model=model, client=client, memory_integration=memory_integration)

In [ ]:
# Query
query = "If you wanted to build an app that lets users track their daily routine and interventions, and use counterfactuals based on other users and counterfactuals using quality research to provide them with causal impact of their interventions (for e.g., you drink black coffee every morning leads to higher energy levels) or also suggests changes to their life based on causal analysis, how would you go about doing it?"

In [ ]:
response, memory_id = pm_agent(query, num_reflections=0)
print(remove_markdown(response))

🔍 Searching the web...
🤔 Reflecting 1 time(s)...
Of course. You've provided my previous strategic breakdown and a very insightful critique of it. This is exactly how a product strategy evolves—from a high-level vision to a battle-tested, tactical plan that anticipates real-world challenges.

My initial response was a solid V1 plan. Your feedback correctly identifies the critical elements that would turn it from a good plan into a great, shippable one. Let's integrate that feedback and create a more robust, actionable V1 execution plan.

Here is the improved, more detailed approach to move V1: The "Smart Diary" from strategy to execution.

From Strategy to Execution: A More Robust Plan for V1 - The "Smart Diary"

Our V1 goal remains the same, but our approach will be more rigorous: "Can we get a specific user group to track their subjective and objective data consistently?" Failure here invalidates the entire product concept, so we must de-risk it ruthlessly.

Here are the concrete, upd